# Potentiel des méthodes de deep learning pour cartographier l’occupation du sol de La Réunion à partir d’images de télédétection

Les dfférentes section de ce NoteBook permettent d'implémenter un réseau de neurones. <br>
Ainsi, après avoir construit le dataset à l'aide d'une image pléaide + polygones de verités terrain il s'agit de préparer cette base de donnée.<br>
Ensuite, il faut définir les métriques utilisées pour evaluer le réseau ainsi que les paramètres du réseau que l'on va faire varier. <br>
Les métriques que l'on fait varier dans ce notebook sont : <br>
- Matrice de confusion <br>
- Accuracy <br>
- Loss <br>

### Important 
   Tous les modules sont à télecharger en temps qu'administrateur.. <br>
   Imports avec quelques problèmes de version : gdal, tf <br>
   Le modèle cnn est fait sur Keras, il est important de rester en Keras pour utiliser le code (et pas "tf.keras")
    
### Rôle de chaque fichier dans le dossier: 

**Dataset** : <br>
    -Image GéoTiff Pléaide <br>
    - Vérité terrain.shp <br>
    -Dossier "Dataset_xx" avec xx la taille de l'imagette <br>

**Fichiers de code** : <br>
    - Base de donées : Le fichier "Creation_Dataset" <br>
    - Main_metrics : ce fichier 
    - Classification : Inférence du cnn. 
    
    
Remarques pour compiler le code : <br>
-si le training ne marche pas, fermer tous les processus pyhton et recommencer <br>
-si le résultat n'est pas celui esperé --> restart Kenerl


#### Sommaire

* [Variables](#chapter0) 
* [Fonctions](#chapter1) 
* [Dataset](#chapter2)
    * [Affichage](#section_1)
    * [Résumé des diccionaires et listes ](#section_1_2)
    * [Gestion de la base de donée](#section_1_3)
* [Implémentation du CNN](#chapter2)
    * [Modèle](#section_2_2)
    * [Training](#section_2_3)

### Variables <a class="anchor" id="chapter0"></a>

> Chemin de la vérité terrain : fichier csv

In [5]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"


In [6]:
datafile_path = 'Dataset_50x50_Niveau2_v9.csv' 
nb_class= 10

**Documentation : Parralèlisme avec Tensorflow / Comet** <br>
https://github.com/comet-ml/comet-examples/blob/master/tensorflow-1/comet-tf1-distributed-mirrored-strategy.py <br>

Comet permet de visualiser les "expériences" lors de la recherche de meilleures combinaisons pour les hyperparamètres. <br>
L'interface permet également de visualiser l'évolution des métriques pour un modèle en particulier. <br>
Clé API : Ult9nmi0NXwIYT8yNAlg17jxP

In [7]:
import comet_ml
from comet_ml import Optimizer
comet_ml.init()

COMET INFO: Comet API key is valid


In [8]:
import sys
import datetime
import math
import itertools
import numpy as np
import numpy
import h5py
import scipy
import io
from datetime import datetime 

#import pandas as pd
from pandas import read_csv
from numpy import zeros
from numpy import ones

from os import listdir
from numpy import asarray
from keras import backend

from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

import sklearn.metrics

from PIL import Image
from scipy import ndimage
import tensorflow as tf
from tensorflow import keras
from tensorflow.python.framework import ops
from matplotlib import pyplot
from matplotlib.image import imread
import matplotlib.ticker as plticker

import csv 

from osgeo import gdal
from osgeo import gdal_array
from osgeo import osr
from numpy import asarray
#from cnn_utils import *
#from pyrsgis import raster

import sklearn.metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import fbeta_score
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
from sklearn.metrics import ConfusionMatrixDisplay


from tensorflow.keras import datasets, layers, models
from keras.layers import Conv2D, MaxPooling2D, Dense, Input, BatchNormalization, GlobalAveragePooling2D, Dropout
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator

from plot_model import plot_model

from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import f1_score

from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.utils import plot_model
from keras.models import model_from_json

#Plusieurs fichiers pour ne pas etouffer le code 
import import_ipynb
#import functions



> Beaucoup d'imports.... mais ca facilite la vie !

In [9]:
mirrored_strategy = tf.distribute.MirroredStrategy()


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


In [10]:
experiment = comet_ml.Experiment(
    project_name="DeepLearning_MOS", 
)
experiment.add_tag("niveau2")


COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/benlamlih/remote-sensing-classif/f1505feb0efa415a8c89205c8be4d4c0



In [11]:
experiment.get_key()

'f1505feb0efa415a8c89205c8be4d4c0'

In [12]:
from comet_ml import Optimizer

>Lors de la recherche d'hyperparamètres, on chosiit l'algortihme Bayésien et on définit les paramètres à faire varier ainsi que leurs plages de valeurs.

In [13]:
# We only need to specify the algorithm and hyperparameters to use:
config = {
    # We pick the Bayes algorithm:
    "algorithm": "bayes",

    # Declare your hyperparameters in the Vizier-inspired format:
    "parameters": {
        "dropout": {"type": "float", "min": 0, "max": 1},
        "nunits" : {"type": "integer", "min": 200, "max": 1000 }
    },

    # Declare what we will be optimizing, and how:
    "spec": {
      "metric": "test_acc",
      "objective": "maximize"
    },
}


In [14]:
opt = Optimizer(config)

COMET INFO: COMET_OPTIMIZER_ID=c3f0a884cba74c828ed4bed09955b47b
COMET INFO: Using optimizer config: {'algorithm': 'bayes', 'configSpaceSize': 'infinite', 'endTime': None, 'id': 'c3f0a884cba74c828ed4bed09955b47b', 'lastUpdateTime': None, 'maxCombo': 0, 'name': 'c3f0a884cba74c828ed4bed09955b47b', 'parameters': {'dropout': {'max': 1, 'min': 0, 'scalingType': 'uniform', 'type': 'float'}}, 'predictor': None, 'spec': {'gridSize': 10, 'maxCombo': 0, 'metric': 'test_acc', 'minSampleSize': 100, 'objective': 'maximize', 'retryAssignLimit': 0, 'retryLimit': 1000}, 'startTime': 17273548909, 'state': {'mode': None, 'seed': None, 'sequence': [], 'sequence_i': 0, 'sequence_pid': None, 'sequence_retry': 0, 'sequence_retry_count': 0}, 'status': 'running', 'suggestion_count': 0, 'trials': 1, 'version': '2.0.1'}


### Fonctions <a class="anchor" id="chapter1"></a>
#### Utilitaires

### Dataset <a class="anchor" id="chapter1"></a>

In [15]:
#On commence par le commencement : Importer la vérité terrain ici de la création de la base de donnée 
#with open(datafile_path, 'r') as f:
    #mapping_csv  = csv.reader(f)
mapping_csv = read_csv(datafile_path)

#Fonction utile pour la suite 
def getElem(file, i, j):
    with open(file, 'r') as f:
        reader = csv.reader(f)
        for line in reader:
            if reader.line_num - 1 == i:
                return line[j]
            
def csvcount(filename):
    with open(filename, 'r') as f:
        i = 0
        for ligne in f:
            i += 1
    return i
lignes= csvcount(datafile_path)
print(lignes)
%store lignes


512974
Stored 'lignes' (int)


### Affichage de quelques images du dataset <a class="anchor" id="section1"></a>

>Dé-commenter les prints pour voir le résultat des fonctions  

>Création de différents dictionnaires à partir du fichier cvs afin de créer une base de donnée homogène quelque soit le cnn choisie. <br>
>La sortie du cnn sera un vecteur ligne de zéros avec un '1' à la position de la classe associée issue de la vérité terrain <br>

### Résumé des diccionaires et listes <a class="anchor" id="section_1_2"></a>
<ul>
<li> Mapping: lier le label et le numero de la classe </li>
<li> file_mapping : lier le nom du fichier et sa classe </li>
<li> Target : vecteur sortie </li>
<li> Labels : noms des classes </li>
<li> Tags : numéro des classes - niveau 3 (en int) </li>
<li> Tags1 : numéro des classes - niveau 3 (en string) </li>

</ul>



In [16]:
# Creation de dicctionaires pour faciliter la lecture de datasets 
#Create a diccionary to link labels and class (fait manuellement)
mapping = {
    "Canne a sucre" : 0,
    "Paturage et fourrage" :1,
    "Maraichage": 2,
    "Arboriculture": 3,
    "Espace boise": 4,
    "Lande et savane":5,
    "Rocher et sol nu naturel":6,
    "Ombre due au relief":7,
    "Eau":8,
    "Espace artificialise":9,
    "Culture sous serre ou ombrage":10,
}

#Create a set of labels 
#boucle sur le [0] du diccionaire 
labels = []
for k,v in mapping.items():
    labels.append(k)
    
#Create a set of tags (class) 
#boucle sur le [0] du diccionaire 
tags = []
for k,v in mapping.items():
    tags.append(int(v))

#listes spéciales pour la matrice de confusion 
tags1 = []
for k,v in mapping.items():
    tags1.append(v)
    
    
# create a mapping of filename to tags 
def create_file_mapping(mapping_csv):
    mapping = dict()
    for i in range(len(mapping_csv)):
        name, tags = mapping_csv['Image FileName'][i], mapping_csv[' Class'][i]
        mapping[name] = tags
    return mapping


#Transform each class into a vector of loads of 0 and a single 1 
def one_hot_encode(tags, mapping):
    # create empty vector
    # mark 1 for each tag in the vector
    encoding = zeros(nb_class, dtype='int8')
    #Modification pour la classe friche qui a le code 200 en 2020 et qui est converti en code 2
    encoding[int(mapping[tags])]=1
            #Verification que le vecteur est bon
    return encoding

file_mapping =create_file_mapping(mapping_csv)




### Gestion de la base de donnée <a class="anchor" id="section_1_3"></a>
Ici on convertir les images 'Tiff' en numpy array pour les donner en entrée au réseau de neurones.<br>
X : liste de matrices <br>
Y : liste de vecteurs lignes <br>

**Modification du 21/05/2021** <br>
Parcourir les 185 sous-répertoires de "Dataset_50x50" et store trainX, testX, trainY, testY pour ne pas refaire la conversion images/numpy array ultèrieurement <br>

**A noter** <br>
Dans un soucis de clarté je note testX_"nom du modèle". Par exemple pour Alexnet les variables seront testX_Alexnet...

> A décommenter pour génerer la base de donnée en matrice

In [17]:
str(datetime.now())

'2021-08-09 21:37:07.360192'

>Pour afficher une matrice sans la limite habituelle de Jupyter Lab

>Lignes magiques : %store pour enregistrer et %store -r pour récuperer 

In [18]:
%store -r trainX
%store -r trainY
%store -r testX
%store -r testY

>Callback pour avoir la matrice de confusion chaque 100 epochs 

In [19]:
from tensorflow.keras.callbacks import Callback
class ConfusionMatrixCallback(Callback):
    def __init__(self, experiment, inputs, targets):
        self.experiment = experiment
        self.inputs = inputs
        self.targets = targets

    def on_epoch_end(self, epoch, logs={}):
        #if epoch%100==0: 
        predicted = self.model.predict(self.inputs)
        self.experiment.log_confusion_matrix(
            self.targets,
            predicted,
            title="Confusion Matrix, Epoch #%d" % (epoch + 1),
            file_name="confusion-matrix-%03d.json" % (epoch + 1),
    )

>Callback pour avoir la matrice de confusion chaque 100 epochs avec des images comme exemple

In [20]:
class SaveBestModel(tf.keras.callbacks.Callback):
    def __init__(self, save_best_metric='val_loss', this_max=False):
        self.save_best_metric = save_best_metric
        self.max = this_max
        if this_max:
            self.best = float('-inf')
        else:
            self.best = float('inf')

    def on_epoch_end(self, epoch, logs=None):
        metric_value = logs[self.save_best_metric]
        if self.max:
            if metric_value > self.best:
                self.best = metric_value
                self.best_weights = self.model.get_weights()

        else:
            if metric_value < self.best:
                self.best = metric_value
                self.best_weights= self.model.get_weights()

In [21]:
save_best_model = SaveBestModel()


### Implémentation du CNN <a class="anchor" id="chapter2"></a>

### Modèle <a class="anchor" id="section_2_2"></a>

>Exemple avec hyperparamètres : le modèle prend comme paramètre l'expérience et chque hyperparamètre est définir comme : "experiment.get_parameter("nom_du_parametre")

In [ ]:
nunits=519
def m3_fusion(experiment):
    in_shape = Input(shape=(25, 25, 4))
    out_shape=11  
    
    conv_1 = Conv2D(experiment.get_parameter("nunits")/2, (7, 7), padding='valid', activation='relu')(in_shape)
    conv_1 = BatchNormalization()(conv_1)  
    
    pool_1 = MaxPooling2D((2, 2), strides=(2, 2), padding='same')(conv_1)
    
    conv_2 = Conv2D(experiment.get_parameter("nunits"), (3, 3), padding='valid', activation='relu')(pool_1)
    conv_2 = BatchNormalization()(conv_2)  
    
    conv_3 = Conv2D(experiment.get_parameter("nunits"), (3, 3), padding='same', activation='relu')(conv_2)
    conv_3 = BatchNormalization()(conv_3)  
    conv_3 = keras.layers.concatenate([conv_2, conv_3])
    
    conv_4 = Conv2D(experiment.get_parameter("nunits"), (1, 1), padding='valid', activation='relu')(conv_3)
    conv_4 = BatchNormalization()(conv_4)  

    flatten= GlobalAveragePooling2D()(conv_4)
    
    dropout = Dropout (experiment.get_parameter("dropout"))(flatten)
    output = Dense(out_shape, activation ='softmax')(dropout)
    
    model = Model(inputs=in_shape, outputs=output)

    #print(model.summary())
    #plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)
    
    model.compile(optimizer='adam', loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model
# define model
#model = m3_fusion()

>Alexnet

>Experimentation : Nouveau modèle


### Training <a class="anchor" id="section_2_3"></a>

In [23]:
callback = ConfusionMatrixCallback(experiment, testX, testY)


In [24]:
str(datetime.now())

'2021-08-09 21:37:14.698740'

In [25]:
def train(experiment, model, x_train, y_train, x_test, y_test):
    model.fit(
        trainX,
        trainY,
        batch_size=32,
        epochs=100,
        callbacks=[callback, save_best_model],
        validation_data=(testX, testY),
    )

In [26]:
def evaluate(experiment, model, x_test, y_test):
    score = model.evaluate(x_test, y_test, verbose=0)
    print(score)


In [27]:
es = tf.keras.callbacks.EarlyStopping( monitor='val_accuracy', min_delta = 0.03, mode ='max', patience=40, restore_best_weights=True)

>Callbacks: Matrice de confusion + early stopping

>Décommenter la boucle "for experiment in opt.get_experiments(project_name="Remote_sensing_classif")"si recherche d'hp

In [28]:
for experiment in opt.get_experiments(project_name="DeepLearning_MOS"):
    # Log parameters, or others:
    #experiment.log_parameter("epochs", 300)

        # Build the model:
    model = m3_fusion(experiment)



            # Train it:
    train(experiment, model, trainX, trainY, testX, testY)


    y_pred = np.argmax(testY, axis=1)
    test = model.predict(testX)
    rounded = np.argmax(test, axis=1)
    f1 = f1_score(y_pred, rounded, average=None)


    precision = precision_score(y_pred, rounded, average=None)
    recall = recall_score(y_pred, rounded, average=None)

    metrics = {"f1": f1,
                    "recall": recall,
                    "precision": precision
                }


    experiment.log_metrics(metrics)

            # How well did it do?
    evaluate(experiment, model, testX, testY)

        # Optionally, end the experiment:

COMET INFO: Ignoring automatic log_parameter('verbose') because 'keras:verbose' is in COMET_LOGGING_PARAMETERS_IGNORE


Epoch 1/100
5611/5611 [==============================] - 296s 52ms/step - loss: 0.8069 - accuracy: 0.7302 - val_loss: 0.5731 - val_accuracy: 0.8094
Epoch 2/100
5611/5611 [==============================] - 294s 52ms/step - loss: 0.4869 - accuracy: 0.8322 - val_loss: 0.7414 - val_accuracy: 0.7832
Epoch 3/100
5611/5611 [==============================] - 294s 52ms/step - loss: 0.3876 - accuracy: 0.8649 - val_loss: 0.9648 - val_accuracy: 0.7048
Epoch 4/100
5611/5611 [==============================] - 294s 52ms/step - loss: 0.3363 - accuracy: 0.8828 - val_loss: 2.5237 - val_accuracy: 0.6445
Epoch 5/100
5611/5611 [==============================] - 295s 53ms/step - loss: 0.3025 - accuracy: 0.8959 - val_loss: 0.4107 - val_accuracy: 0.8562
Epoch 6/100
5611/5611 [==============================] - 293s 52ms/step - loss: 0.2727 - accuracy: 0.9059 - val_loss: 0.4046 - val_accuracy: 0.8831
Epoch 7/100
5611/5611 [==============================] - 294s 52ms/step - loss: 0.2535 - accuracy: 0.9123 - val_

COMET WARNING: Cannot safely convert array([0.90034825, 0.94079127, 0.72429332, 0.86542516, 0.91495601,
       0.88317142, 0.97793737, 0.97542019, 0.9905738 , 0.95369458]) object to a scalar value, using its string representation for logging.
COMET WARNING: Cannot safely convert array([0.92411328, 0.91556028, 0.72697368, 0.90618102, 0.92180972,
       0.87621805, 0.97743115, 0.97381183, 0.99113812, 0.9673551 ]) object to a scalar value, using its string representation for logging.
COMET WARNING: Cannot safely convert array([0.87777488, 0.9674523 , 0.72163265, 0.82817754, 0.90820347,
       0.89023603, 0.97844411, 0.97703387, 0.99001012, 0.94041451]) object to a scalar value, using its string representation for logging.


[2.240379810333252, 0.9310945272445679]


In [29]:
str(datetime.now())

'2021-08-10 06:40:05.820800'

>Enregistrer le modèle

In [31]:
#Implement early stopping to avoid over-fitting
#es = EarlyStes = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=50)opping(monitor='val_loss', mode='min', verbose=1)

In [32]:
experiment.end()

COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/benlamlih/remote-sensing-classif/f1505feb0efa415a8c89205c8be4d4c0
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [100]                  : (0.776751697063446, 0.9937451481819153)
COMET INFO:     batch_accuracy [56200]          : (0.0625, 1.0)
COMET INFO:     batch_loss [56200]              : (0.00016871656407602131, 2.4661829471588135)
COMET INFO:     epoch_duration [100]            : (322.8439999999973, 327.8590000000004)
COMET INFO:     f1                              : [0.90034825 0.94079127 0.72429332 0.86542516 0.91495601 0.88317142
 0.97793737 0.97542019 0.9905738  0.95369458]
COMET INFO:     loss [100]                      : (0.019582241773605347, 0.6607242822647095)
COMET INFO:     precision                       : [0.92